In [3]:
import requests
import os
import json
import re
import pandas as pd
from config import TOKEN

bearer_token = TOKEN

# Create dictionary of twitter accounts. (These can be changed. I just selected these for the sake of testing the script.)
users = {'telegraph':16343974,
         'cbcnews':6433472,
         'ap':51241574,
         'washingtonpost':2467791,
         'wsj':3108351,
         'nytimes':807095,
         'cnn':759251}

# Initialize empty dataframe
final_df = pd.DataFrame(columns=['source','tweet'])

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r

# Loop through dictionary of accounts
for user in users:
    
    # define query parameters
    params = {"max_results":5,
             'exclude':'replies,retweets'}
    # create request URL
    URL = "https://api.twitter.com/2/users/{}/tweets".format(users[user])
    response = requests.request("GET", URL, auth=bearer_oauth, params = params).json()
    
    # loop through payload to grab individual tweets for each account
    for x in range(5):
        # grab tweet text
        tweet = response['data'][x]['text']
        # removed URL from end of tweet and strip leading/trailing spaces
        clean_tweet = re.sub(r'https://.*', '',tweet).strip()
        source = user
        tweet = clean_tweet
        # user account name and tweet test to temporary dataframe.
        temp_df = pd.DataFrame([[source, tweet]], columns =['source','tweet'])
        # add temporary dataframe to final dataframe.
        final_df = pd.concat([final_df,temp_df],ignore_index=True)
final_df


,source,tweet
0,telegraph,"✍️""Now the world is rediscovering its love of ..."
1,telegraph,Rishi Sunak has threatened oil and gas giants ...
2,telegraph,🔴Ukraine's Defence Minister Oleksiy Reznikov h...
3,telegraph,💼People who shared the same space came up with...
4,telegraph,Muhammad Saleh's uncle denies the family refus...
5,cbcnews,People in B.C. who have seen fracking sites se...
6,cbcnews,A Federal Court judge has ordered two airports...
7,cbcnews,B.C. Ferries has halted service between Prince...
8,cbcnews,#Opinion: As health-care delivery becomes more...
9,cbcnews,Investigations are underway into a January fra...


In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

tweets = final_df['tweet']

In [7]:
tweet_scores = []
for tweet in tweets:
    score = analyser.polarity_scores(tweet)
    tweet_scores.append(score)
    
#Converting List of Dictionaries into Dataframe
tweetScore_df= pd.DataFrame(tweet_scores)
tweetScore_df.head(20)

,neg,neu,pos,compound
0,0.105,0.796,0.098,0.1779
1,0.142,0.677,0.181,0.2023
2,0.389,0.572,0.038,-0.9167
3,0.000,0.882,0.118,0.3400
4,0.333,0.667,0.000,-0.6124
5,0.053,0.868,0.079,0.1779
6,0.268,0.732,0.000,-0.7717
7,0.000,0.786,0.214,0.5994
8,0.000,1.000,0.000,0.0000
9,0.000,0.899,0.101,0.4019


In [9]:
sentiment = tweetScore_df['compound'].to_list()
final_df['sentiment_score'] = sentiment
final_df

,source,tweet,sentiment_score
0,telegraph,"✍️""Now the world is rediscovering its love of ...",0.1779
1,telegraph,Rishi Sunak has threatened oil and gas giants ...,0.2023
2,telegraph,🔴Ukraine's Defence Minister Oleksiy Reznikov h...,-0.9167
3,telegraph,💼People who shared the same space came up with...,0.3400
4,telegraph,Muhammad Saleh's uncle denies the family refus...,-0.6124
5,cbcnews,People in B.C. who have seen fracking sites se...,0.1779
6,cbcnews,A Federal Court judge has ordered two airports...,-0.7717
7,cbcnews,B.C. Ferries has halted service between Prince...,0.5994
8,cbcnews,#Opinion: As health-care delivery becomes more...,0.0000
9,cbcnews,Investigations are underway into a January fra...,0.4019


In [16]:
final_df.iloc[2]

source                                                     telegraph
tweet              🔴Ukraine's Defence Minister Oleksiy Reznikov h...
sentiment_score                                              -0.9167
Name: 2, dtype: object

In [15]:
final_df.iloc[2]['tweet']

'🔴Ukraine\'s Defence Minister Oleksiy Reznikov has warned the country has "extremely difficult weeks" ahead, warning of major "destruction" in a developing Russian offensive in the east'